In [ ]:
import pandas as pd
msg=pd.read_csv('SMSSpamCollection.txt',sep='\t',names=["Label","Message"])

In [3]:
msg

,Label,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [4]:
msg.shape

(5572, 2)

In [5]:
import re
import nltk
nltk.download ('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     c:\Users\huzef\anaconda3\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [7]:
corpus=[]
for i in range(0,len(msg)):
    review=re.sub('[^a-zA-Z0-9]',' ',msg['Message'][i])
    review=review.lower()
    review=review.split()
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)
corpus

KeyboardInterrupt: 

BOW

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(binary=True,max_features=2500)

In [24]:
X=cv.fit_transform(corpus).toarray()

In [25]:
X.shape

(5572, 2500)

In [44]:
y=pd.get_dummies(msg['Label'],dtype=int)
y=y.iloc[:,1].values
y

array([0, 0, 1, ..., 0, 0, 0])

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [29]:
from sklearn.naive_bayes import MultinomialNB
spam=MultinomialNB().fit(x_train,y_train)# training the model directly

In [30]:
y_pred=spam.predict(x_test)

In [31]:
from sklearn.metrics import accuracy_score,classification_report

In [32]:
accuracy_score(y_test,y_pred)

0.9838565022421525

In [33]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       966
           1       0.94      0.94      0.94       149

    accuracy                           0.98      1115
   macro avg       0.97      0.97      0.97      1115
weighted avg       0.98      0.98      0.98      1115



TF_IDF

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [39]:
tv=TfidfVectorizer()
x=tv.fit_transform(corpus).toarray()

In [40]:
from sklearn.model_selection import train_test_split

In [41]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [43]:
from sklearn.naive_bayes import MultinomialNB
spam_tf=MultinomialNB().fit(x_train,y_train)

In [44]:
y_pred=spam_tf.predict(x_test)

In [45]:
accuracy_score(y_test,y_pred)

0.9713004484304932

In [46]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       1.00      0.97      0.98       998
           1       0.79      1.00      0.88       117

    accuracy                           0.97      1115
   macro avg       0.89      0.98      0.93      1115
weighted avg       0.98      0.97      0.97      1115



Using Random forest

In [47]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()
classifier.fit(x_train,y_train)

RandomForestClassifier()

In [48]:
y_pred=classifier.predict(x_test)

In [49]:
accuracy_score(y_test,y_pred)

0.9802690582959641

In [51]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99       988
           1       0.85      1.00      0.92       127

    accuracy                           0.98      1115
   macro avg       0.93      0.99      0.95      1115
weighted avg       0.98      0.98      0.98      1115



WORD2VEC Implementation

AVERAGE WORD2VEC


In [ ]:
import gensim.downloader as api
wv=api.load('word2vec-google-news-300')

In [9]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [10]:
corpus=[]
for i in range(0,len(msg)):
    review=re.sub('[^a-zA-Z0-9]',' ',msg['Message'][i])
    review=review.lower()
    review=review.split()
    review=[lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)
corpus

['go jurong point crazy available bugis n great world la e buffet cine got amore wat',
 'ok lar joking wif u oni',
 'free entry 2 wkly comp win fa cup final tkts 21st may 2005 text fa 87121 receive entry question std txt rate c apply 08452810075over18',
 'u dun say early hor u c already say',
 'nah think go usf life around though',
 'freemsg hey darling 3 week word back like fun still tb ok xxx std chgs send 1 50 rcv',
 'even brother like speak treat like aid patent',
 'per request melle melle oru minnaminunginte nurungu vettam set callertune caller press 9 copy friend callertune',
 'winner valued network customer selected receivea 900 prize reward claim call 09061701461 claim code kl341 valid 12 hour',
 'mobile 11 month u r entitled update latest colour mobile camera free call mobile update co free 08002986030',
 'gonna home soon want talk stuff anymore tonight k cried enough today',
 'six chance win cash 100 20 000 pound txt csh11 send 87575 cost 150p day 6days 16 tsandcs apply reply

In [11]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess


In [12]:
corpus[0]

'go jurong point crazy available bugis n great world la e buffet cine got amore wat'

In [15]:
words=[]
for sent in corpus:
    sent_token=sent_tokenize(sent)
    for sent in sent_token:
        words.append(simple_preprocess(sent))# simple_preprocess:- convert a document into a list of lowercase token (lowercase)
words

[['go',
  'jurong',
  'point',
  'crazy',
  'available',
  'bugis',
  'great',
  'world',
  'la',
  'buffet',
  'cine',
  'got',
  'amore',
  'wat'],
 ['ok', 'lar', 'joking', 'wif', 'oni'],
 ['free',
  'entry',
  'wkly',
  'comp',
  'win',
  'fa',
  'cup',
  'final',
  'tkts',
  'st',
  'may',
  'text',
  'fa',
  'receive',
  'entry',
  'question',
  'std',
  'txt',
  'rate',
  'apply',
  'over'],
 ['dun', 'say', 'early', 'hor', 'already', 'say'],
 ['nah', 'think', 'go', 'usf', 'life', 'around', 'though'],
 ['freemsg',
  'hey',
  'darling',
  'week',
  'word',
  'back',
  'like',
  'fun',
  'still',
  'tb',
  'ok',
  'xxx',
  'std',
  'chgs',
  'send',
  'rcv'],
 ['even', 'brother', 'like', 'speak', 'treat', 'like', 'aid', 'patent'],
 ['per',
  'request',
  'melle',
  'melle',
  'oru',
  'minnaminunginte',
  'nurungu',
  'vettam',
  'set',
  'callertune',
  'caller',
  'press',
  'copy',
  'friend',
  'callertune'],
 ['winner',
  'valued',
  'network',
  'customer',
  'selected',
  're

In [16]:
import gensim
model=gensim.models.Word2Vec(words,window=5,min_count=2)

In [17]:
model.wv.index_to_key

['call',
 'get',
 'ur',
 'gt',
 'lt',
 'go',
 'ok',
 'day',
 'free',
 'know',
 'come',
 'like',
 'good',
 'time',
 'got',
 'text',
 'love',
 'want',
 'send',
 'need',
 'one',
 'txt',
 'today',
 'going',
 'stop',
 'home',
 'lor',
 'sorry',
 'see',
 'still',
 'take',
 'mobile',
 'back',
 'da',
 'reply',
 'dont',
 'think',
 'tell',
 'week',
 'phone',
 'hi',
 'new',
 'later',
 'please',
 'pls',
 'co',
 'msg',
 'dear',
 'make',
 'night',
 'message',
 'say',
 'well',
 'min',
 'thing',
 'much',
 'claim',
 'great',
 'hope',
 'oh',
 'hey',
 'give',
 'number',
 'happy',
 'work',
 'friend',
 'wat',
 'yes',
 'way',
 'www',
 'let',
 'prize',
 'right',
 'tomorrow',
 'already',
 'ask',
 'said',
 'win',
 'life',
 'amp',
 'cash',
 'yeah',
 'im',
 'really',
 'tone',
 'babe',
 'meet',
 'find',
 'miss',
 'morning',
 'service',
 'last',
 'uk',
 'thanks',
 'care',
 'would',
 'com',
 'anything',
 'year',
 'lol',
 'also',
 'nokia',
 'every',
 'feel',
 'keep',
 'sure',
 'pick',
 'sent',
 'urgent',
 'contact',


In [20]:
model.corpus_count

5565

In [21]:
model.epochs

5

In [23]:
model.wv.similar_by_word('kid')

[('also', 0.9966513514518738),
 ('bt', 0.9965690970420837),
 ('special', 0.9964936375617981),
 ('since', 0.9964811205863953),
 ('detail', 0.9964641332626343),
 ('chat', 0.9964593648910522),
 ('give', 0.9964571595191956),
 ('time', 0.9964353442192078),
 ('want', 0.9964260458946228),
 ('always', 0.9964019656181335)]

In [24]:
model.wv['kid'].shape

(100,)

In [25]:
import numpy as np

In [30]:
def avg_word2vec(doc):
    #remove out of vocabulary words
    #sent=[word for in doc if word in model.wwv.index_to_key]
    #print(sent)
    #[np.zeros(len(model.wv.index_to_key)),axis=0]
    return np.mean([model.wv[word]for word in doc if word in model.wv.index_to_key],axis=0 )

In [31]:
!pip install tqdm

In [32]:
from tqdm import tqdm
words[73]

['performed']

In [33]:
X=[]
for i in tqdm(range(len(words))):
    X.append(avg_word2vec(words[i]))

  0%|          | 0/5565 [00:00<?, ?it/s]c:\Users\huzef\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\huzef\anaconda3\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 5565/5565 [00:01<00:00, 2850.91it/s]


In [50]:
x_new=np.array(X)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (5565,) + inhomogeneous part.

In [42]:
from sklearn.model_selection import train_test_split

In [46]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

ValueError: Found input variables with inconsistent numbers of samples: [5565, 5572]